The pipes are arranged in a two-dimensional grid of tiles:

- | is a vertical pipe connecting north and south.
- \- is a horizontal pipe connecting east and west.
- L is a 90-degree bend connecting north and east.
- J is a 90-degree bend connecting north and west.
- 7 is a 90-degree bend connecting south and west.
- F is a 90-degree bend connecting south and east.
- . is ground; there is no pipe in this tile.
- S is the starting position of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.

In [5]:
import aoc_helpers
import re

actual = aoc_helpers.load_data('actual.txt')
for line in range(0,len(actual)):
    actual[line] = [i for i in actual[line]]

# Tests

In [9]:
test_1 = [
    ['-','L','|','F','7'],
    ['7','S','-','7','|'],
    ['L','|','7','|','|'],
    ['-','L','-','J','|'],
    ['L','|','-','J','F'],
]

test_2 = [
    ['7','-','F','7','-'],
    ['.','F','J','|','7'],
    ['S','J','L','L','7'],
    ['|','F','-','-','J'],
    ['L','J','.','L','J'],
]

# Code

In [11]:
def find_start(input):
    found = False
    row = 0
    while not found:
        try:
            col = input[row].index('S')
            found = True
        except:
            row+=1

    return (row,col)

print(find_start(test_1)) #(1,1)
print(find_start(test_2)) #(2,0)
print(find_start(actual)) #(2,0)

(1, 1)
(2, 0)
(42, 25)


In [12]:
#coords = (row,column)

conn_map ={
    '|': [( 1, 0),(-1, 0)],
    '-': [( 0, 1),( 0,-1)],
    'L': [( 0, 1),(-1, 0)],
    'J': [( 0,-1),(-1, 0)],
    '7': [( 0,-1),( 1, 0)],
    'F': [( 1, 0),( 0, 1)],
}

In [44]:
def find_pipe(input):
    start_coord = find_start(input)

    pipe = [
        ('S', start_coord)
    ]

    # start by traversing cardinal points to find the first 
    # valid pipe connection
    cardinal_points = {
        (start_coord[0]-1, start_coord[1]):['|', 'F', '7'], 
        (start_coord[0]+1, start_coord[1]):['|', 'L', 'J'],
        (start_coord[0], start_coord[1]-1):['-', 'L', 'F'],
        (start_coord[0], start_coord[1]+1):['-', '7', 'J']
    }

    for point,symbols in cardinal_points.items():
        if input[point[0]][point[1]] in symbols:
            #valid connection, so add to pipe, and continue
            pipe.append((input[point[0]][point[1]], point))
            break

    # then loop through pipe until we get back to 'S'
    counter = 1
    complete = False

    while not complete:
        last = pipe[counter-1][1]
        curr = pipe[counter]
        opts = [
            (
                curr[1][0]+inc[0],
                curr[1][1]+inc[1]
            )
            for inc in conn_map[curr[0]]
        ]

        nxt = [c for c in opts if c!=last][0]
        nxt = (
            input[nxt[0]][nxt[1]],
            nxt
        )

        if nxt[0]=='S':
            complete=True
        else:

            pipe.append(nxt)
            counter+=1

    return pipe

In [49]:
pipe = find_pipe(actual)
print("PART 1: ", int(len(pipe)/2))

PART 1:  7173


In [56]:
def get_inside(input, pipe):

    max_row = len(input)
    max_col = len(input[0])

    count_inside = 0

    for row, line in enumerate(input):
        for col, char in enumerate(line):
            
            #Skip vertices which are part of our pipe
            if (char, (row, col)) in pipe:
                continue
            
            intercept_count = 0
            r2, c2 = row, col

            #Traverse the map in a diagonal
            # We can ignore '7' and 'L' vertices, as our 
            # diagonal hits the apex of the corner
            while r2 < max_row and c2 < max_col:
                char2 = input[r2][c2]
                #First run came out too high - suspect 'S' is actually a '7' or 'L'
                if ((char2, (r2, c2)) in pipe) and \
                    (char2 not in ['7', 'L', 'S']):
                    intercept_count+=1

                r2+=1
                c2+=1
            
            if intercept_count%2==1:
                count_inside+=1

    return count_inside
            

In [54]:
p2_test = [
    '..........',
    '.S------7.',
    '.|F----7|.',
    '.||OOOO||.',
    '.||OOOO||.',
    '.|L-7F-J|.',
    '.|II||II|.',
    '.L--JL--J.',
    '..........',
]

for line in range(0,len(p2_test)):
    p2_test[line] = [i for i in p2_test[line]]

p2_test_pipe = find_pipe(p2_test)

%timeit get_inside(p2_test, p2_test_pipe)


392 µs ± 23.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [57]:
get_inside(actual, pipe)

291